这个就不是一个非常通用的功能了，毕竟VQE自身决定了是可以选择不同线路的

VQE的根本就是变分法，也就是通过修改参数，例如最优化自由能（自由能最小在物理上是对应一个热平衡态），也许可以最小化能量等物理量 获得不同的物理

这里的VQE代码是优化自由能

且我删除了这里的线路，可以参考使用上面的quantum neural network

如果需要控制初态，可以选择使用circ_initial_state功能

这里用了类的数据类型，使用方法请自行bing搜索

我这里实现了多次重复优化并自动拾取最优的结果，供参考用

In [ ]:
import qibo
from qibo import Circuit, gates
from qibo import hamiltonians
from qibo.symbols import X, Y, Z, I
from qibo.noise import NoiseModel, PauliError, DepolarizingError
from qibo.result import MeasurementOutcomes
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm, logm
from scipy.optimize import minimize
from scipy.special import comb
from itertools import combinations
import multiprocessing
from math import pi
qibo.set_backend("numpy")
from numpy.linalg import eigh

In [ ]:
class VQE(object):
    def __init__(self,temperature,layers,n_flip):
        """本类里面的theta是E_i前面的系数，alpha是线路参数"""
        self.temperature = temperature
        self.layers = layers
        self.n_flip = n_flip

    def temp_circ(self,circuit,alpha,flip_index:list):
        """计算的基本线路，得到最终结果需要多个基本线路通过一定比例混合而成"""
        pass # pass是用于占位（使得代码不报错），没有其他功能
    
    def theta_energy_entropy(self,alpha):
        """返回三个值，第一个是theta_list，第二个是能量，第三个是熵"""
        energy = 0
        entropy = 0
        theta_list = []
        qubit_list = list(range(n_qubit))
        for f in range(self.n_flip): # 例：n_flip=2，选取|0101>+所有翻转1个的初态
            temp = list(combinations(qubit_list,f)) # 制备所有可能的翻转可能
            for n in temp: # 具体翻转哪几个qubit
                circ = Circuit(nqubits=n_qubit,density_matrix=False)
                self.temp_circ(circuit=circ,alpha=alpha,flip_index=n)
                state = circ.execute().state()
                E_i = ham.expectation(state=state) # E_i

                theta_i = np.exp(-E_i/self.temperature) # 没有归一化的theta
                theta_list.append(theta_i)
                energy += E_i * theta_i
        normal = sum(theta_list) # 归一化
        energy = energy/normal
        theta_list = theta_list/normal
        for theta in theta_list:
            entropy -= theta * np.log(theta)
        return theta_list,energy,entropy
    
    def free_energy(self,alpha):
        theta_list,energy,entropy = self.theta_energy_entropy(alpha=alpha)
        return energy - self.temperature * entropy
    
    def Opt(self,n_Opt): # 最小化自由能
        """n_Opt：优化次数，取最好的优化结果"""
        Opt_params = np.random.random(3*n_qubit*self.layers)
        Opt_res = minimize(fun=self.free_energy,x0=Opt_params,method='BFGS',tol=1e-03)
        print('free energy=',Opt_res.fun)
        for i in range(1,n_Opt):
            print('Opt:',i)
            Opt_params = np.random.random(3*n_qubit*self.layers)
            temp_res = minimize(fun=self.free_energy,x0=Opt_params,method='BFGS',tol=1e-03)
            if Opt_res.fun-temp_res.fun>1e-05:
                print('Opt update:free energy=',temp_res.fun)
                Opt_res = temp_res
        print(Opt_res.message,Opt_res.fun)
        return Opt_res
